In [2]:
import os
import uuid
import numpy as np
from PIL import Image  # Import the Image class from the PIL (Pillow) library
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [11]:
train_dir = "isec-ic\\dataset\\train"
validation_dir = "isec-ic\\dataset\\valid"
test_dir = "isec-ic\\dataset\\test"

current_dir = os.path.dirname(os.path.realpath(__file__ if '__file__' in locals() else os.getcwd()))

train_dir = os.path.join(current_dir, train_dir)
validation_dir = os.path.join(current_dir, validation_dir)
test_dir = os.path.join(current_dir, test_dir)

# "scab", "rot", "rust", "mildew","scorch",
categories = ["healthy", "blight", "spot",  "mold", "virus", "mite"]


In [12]:
def load_images_from_folder(folder):
    images = []
    labels = []
    subfolders = os.listdir(folder)  # list of subfolders in the main folder
    
    for subfolder in subfolders:
        category = subfolder.split("_")[-1].lower()  # category of the subfolder (e.g., "scab" from "Apple___Apple_scab")
        
        if category in categories:
            print("Loading images from category: " + category + " and subfolder: " + subfolder)
            subfolder_path = os.path.join(folder, subfolder)
            
            for filename in os.listdir(subfolder_path):
                img = Image.open(os.path.join(subfolder_path, filename))
                img = img.resize((224, 224))  # resize if you want to use a different input size
                img = np.array(img) / 255.0  # normalization of the pixel values between 0 and 1
                images.append(img)
                labels.append(categories.index(category))  # index of the category in the list of categories
        else:
            print("Category: " + category + " from subfolder " + subfolder + " is not in the list of categories.")

    return np.array(images), np.array(labels)

In [13]:
x_train, y_train = load_images_from_folder(train_dir)
print("Finished to load the train images")

Loading images from category: spot and subfolder: Tomato___Bacterial_spot
Loading images from category: blight and subfolder: Tomato___Early_blight
Loading images from category: healthy and subfolder: Tomato___healthy
Loading images from category: blight and subfolder: Tomato___Late_blight
Loading images from category: mold and subfolder: Tomato___Leaf_Mold
Loading images from category: spot and subfolder: Tomato___Septoria_leaf_spot
Loading images from category: mite and subfolder: Tomato___Spider_mites Two-spotted_spider_mite
Loading images from category: spot and subfolder: Tomato___Target_Spot
Loading images from category: virus and subfolder: Tomato___Tomato_mosaic_virus
Loading images from category: virus and subfolder: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Finished to load the train images


In [15]:
x_validation, y_validation = load_images_from_folder(validation_dir)
print("Finished to load the validation images")

Loading images from category: spot and subfolder: Tomato___Bacterial_spot
Loading images from category: blight and subfolder: Tomato___Early_blight
Loading images from category: healthy and subfolder: Tomato___healthy
Loading images from category: blight and subfolder: Tomato___Late_blight
Loading images from category: mold and subfolder: Tomato___Leaf_Mold
Loading images from category: spot and subfolder: Tomato___Septoria_leaf_spot
Loading images from category: mite and subfolder: Tomato___Spider_mites Two-spotted_spider_mite
Loading images from category: spot and subfolder: Tomato___Target_Spot
Loading images from category: virus and subfolder: Tomato___Tomato_mosaic_virus
Loading images from category: virus and subfolder: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Finished to load the validation images


In [29]:
x_test, y_test = load_images_from_folder(test_dir)
print("Finished to load the test images");

Loading images from category: blight and subfolder: tomato_blight
Loading images from category: healthy and subfolder: tomato_healthy
Loading images from category: mite and subfolder: tomato_mite
Loading images from category: mold and subfolder: tomato_mold
Loading images from category: spot and subfolder: tomato_spot
Loading images from category: virus and subfolder: tomato_yellow_virus
Finished to load the test images


In [31]:
# Display the total number of images loaded
print("Number of training images: " + str(len(x_train)))
print("Number of validation images: " + str(len(x_validation)))
print("Number of test images: " + str(len(x_test)))

Number of training images: 18345
Number of validation images: 4585
Number of test images: 43


In [37]:
print("Creating the MLP network")
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(224, 224, 3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
print("Finished creating the MLP Model")

Creating the MLP network
Finished creating the MLP Model


In [38]:
print("Starting to train and validate the model")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_data=(x_validation, y_validation))
print("Finished training and validating the model")

Starting to train and validate the model
Epoch 1/10
574/574 [==============================] - 40s 68ms/step - loss: 4.1362 - accuracy: 0.4007 - val_loss: 1.8172 - val_accuracy: 0.5320
Epoch 2/10
574/574 [==============================] - 36s 63ms/step - loss: 1.7983 - accuracy: 0.5010 - val_loss: 1.4746 - val_accuracy: 0.5012
Epoch 3/10
574/574 [==============================] - 37s 64ms/step - loss: 1.3274 - accuracy: 0.5509 - val_loss: 1.3125 - val_accuracy: 0.5208
Epoch 4/10
574/574 [==============================] - 37s 64ms/step - loss: 1.2414 - accuracy: 0.5607 - val_loss: 1.4249 - val_accuracy: 0.5272
Epoch 5/10
574/574 [==============================] - 37s 64ms/step - loss: 1.1186 - accuracy: 0.5869 - val_loss: 1.3601 - val_accuracy: 0.5285
Epoch 6/10
574/574 [==============================] - 37s 64ms/step - loss: 1.0645 - accuracy: 0.6056 - val_loss: 1.0287 - val_accuracy: 0.6124
Epoch 7/10
574/574 [==============================] - 37s 65ms/step - loss: 1.0485 - accuracy: 

In [39]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(y_pred_classes)
print(categories)
confusion = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix: ")
print(confusion)

classification = classification_report(y_test, y_pred_classes, target_names=categories)
print("Classification Report: ")
print(classification)

2/2 [==============================] - 0s 4ms/step
[1 1 2 2 1 5 2 0 0 0 5 5 5 5 5 5 5 5 5 2 2 2 1 2 1 2 3 2 2 2 2 2 0 1 2 2 2
 4 4 2 2 2 4]
['healthy', 'blight', 'spot', 'mold', 'virus', 'mite']
Confusion Matrix: 
[[3 0 1 0 0 0]
 [0 3 2 0 0 1]
 [1 1 7 0 0 0]
 [0 2 4 1 0 0]
 [0 0 3 0 3 0]
 [0 0 2 0 0 9]]
Classification Report: 
              precision    recall  f1-score   support

     healthy       0.75      0.75      0.75         4
      blight       0.50      0.50      0.50         6
        spot       0.37      0.78      0.50         9
        mold       1.00      0.14      0.25         7
       virus       1.00      0.50      0.67         6
        mite       0.90      0.82      0.86        11

    accuracy                           0.60        43
   macro avg       0.75      0.58      0.59        43
weighted avg       0.75      0.60      0.60        43



In [42]:
# generate a new uuid for the model
model_uuid = uuid.uuid4()
# save the model
model.save("models/" + str(model_uuid) + ".keras")